# ACO on ATSP instances


### **Imports**

In [5]:
# Sistema de archivos
import os

# DataFrames y tablas
import pandas as pd

# Visualización en Jupyter
from IPython.display import display

# Combinaciones (para Wilcoxon)
from itertools import combinations

# Módulos que implementamos nosotros
from modules.tsp_utils import read_tsplib
from modules.experiment_utils import (
    run_multiple_experiments,
    summarize_results,
    wilcoxon_rank_sum
)

### **Upload Instances**

In [7]:
# Directory containing all TSPLIB instances for the experiments
instances_dir = "instances"

# List to store all parsed instances
instances_list = []

# Scan the directory and load any file that ends with .atsp
for filename in os.listdir(instances_dir):
    if filename.lower().endswith(".atsp"):
        full_path = os.path.join(instances_dir, filename)

        # Parse TSPLIB file (name, dimension, matrix, etc.)
        data = read_tsplib(full_path)

        if data["name"] == "br17":
            data["n_iterations"] = 150
        elif data["name"] == "ft53":
            data["n_iterations"] = 100
        elif data["name"] == "kro124p":
            data["n_iterations"] = 50
            
        # Store essential fields in a clean structure
        instances_list.append({
            "name": data["name"],               # instance label (e.g., br17)
            "path": full_path,                  # absolute path to the file
            "dimension": data["dimension"],     # number of cities
            "distance_matrix": data["distance_matrix"],
            "n_iterations": data["n_iterations"]
        })

# Report which instances were loaded (useful sanity check)
for inst in instances_list:
    print(f"Loaded {inst['name']} ({inst['dimension']} cities) from {inst['path']}")


Loaded br17 (17 cities) from instances\br17.atsp
Loaded ft53 (53 cities) from instances\ft53.atsp
Loaded kro124p (100 cities) from instances\kro124p.atsp


In [8]:
# ===========================================================
# ACO parameter configurations
# -----------------------------------------------------------
# Each configuration represents a different balance between:
# - pheromone influence (alpha)
# - heuristic influence (beta)
# - evaporation rate (rho)
#
# The field n_ants="use_n_cities" is a placeholder: it will be
# replaced later with the actual number of cities for each instance.
# ===========================================================

configs = {
    'config_1': {
        # Strong heuristic guidance (higher beta) and relatively fast evaporation.
        'alpha': 1.0,
        'beta': 5.0,
        'rho': 0.5,
        'n_ants': "use_n_cities",
        'n_iterations': "use_n_iterations",
        'q': 1.0
    },
    'config_2': {
        # More pheromone-driven behavior (higher alpha) with slower evaporation.
        'alpha': 2.0,
        'beta': 2.0,
        'rho': 0.3,
        'n_ants': "use_n_cities",
        'n_iterations': "use_n_iterations",
        'q': 1.0
    },
    'config_3': {
        # More exploratory setting (lower alpha and beta) with high evaporation.
        'alpha': 1.0,
        'beta': 1.0,
        'rho': 0.7,
        'n_ants': "use_n_cities",
        'n_iterations': "use_n_iterations",
        'q': 1.0
    },
}

# ===========================================================
# Best-known values for each TSPLIB instance used.
# These are included for evaluation and statistical analysis.
# ===========================================================

BEST_KNOWN = {
    "br17": 39,
    "ft53": 6905,      # Typical TSPLIB reference value
    "kro124": 36230,   # Typical TSPLIB reference value
}

# Number of independent runs required by the assignment
N_RUNS = 31


In [9]:
def run_aco_for_instance(instance, configs, best_known_dict, n_runs=31, base_seed=42):
    """
    Run ACO experiments for a single TSPLIB instance.

    Parameters
    ----------
    instance : dict
        Dictionary with keys: 
            - name
            - distance_matrix
            - dimension
            - n_iterations  (custom per instance)
    configs : dict
        Dictionary of ACO parameter configurations.
    best_known_dict : dict
        Maps instance name to best-known value (if available).
    n_runs : int
        Number of independent runs per configuration.
    base_seed : int
        Base seed for reproducibility.

    Returns
    -------
    pd.DataFrame
        Table with aggregated results for this instance.
    """
    name = instance["name"]
    dist_matrix = instance["distance_matrix"]
    n_cities = instance["dimension"]
    n_iterations = instance["n_iterations"]
    
    best_known = best_known_dict.get(name, None)

    print("\n===================================================")
    print(f"               RUNNING INSTANCE: {name.upper()}")
    print("===================================================")

    rows = []

    for cfg_name, params in configs.items():

        # Copy parameters and adapt number of ants and iterations
        cfg = params.copy()
        if cfg["n_ants"] == "use_n_cities":
            cfg["n_ants"] = n_cities
        cfg["n_iterations"] = n_iterations

        print(f"\n--- Running {cfg_name} ---")

        res = run_multiple_experiments(
            distance_matrix=dist_matrix,
            instance_name=name,
            config_name=cfg_name,
            n_runs=n_runs,
            aco_params=cfg,
            base_seed=base_seed,
        )

        lengths = res["lengths"]
        tours = res["tours"]

        summary = summarize_results(lengths, best_known=best_known)

        best_idx = lengths.index(min(lengths))
        best_len = lengths[best_idx]
        best_tour = tours[best_idx]

        # Store row
        rows.append({
            "instance": name,
            "config": cfg_name,
            "n_cities": n_cities,
            "mean": summary["mean"],
            "median": summary["median"],
            "std": summary["std"],
            "min": summary["min"],
            "max": summary["max"],
            "hits_best_known": summary.get("hits_best_known", None),
            "best_length": best_len,
            "best_tour": best_tour,
        })

        # Print summary
        print("Summary:")
        for k, v in summary.items():
            print(f"  {k}: {v}")
        print("\nBest run:")
        print(f"  Best length = {best_len}")
        print(f"  Best tour   = {best_tour}")

    return pd.DataFrame(rows)

In [10]:
# === RUN BR17 ===

inst_br17 = next(inst for inst in instances_list if inst["name"].lower() == "br17")

df_br17 = run_aco_for_instance(
    instance=inst_br17,
    configs=configs,
    best_known_dict=BEST_KNOWN,
    n_runs=N_RUNS,
    base_seed=42
)

display(df_br17)


               RUNNING INSTANCE: BR17

--- Running config_1 ---
Summary:
  n: 31
  mean: 39.0
  median: 39.0
  std: 0.0
  min: 39.0
  max: 39.0
  hits_best_known: 31

Best run:
  Best length = 39.0
  Best tour   = [5, 15, 14, 6, 3, 4, 8, 16, 7, 9, 10, 1, 12, 2, 13, 0, 11]

--- Running config_2 ---
Summary:
  n: 31
  mean: 39.0
  median: 39.0
  std: 0.0
  min: 39.0
  max: 39.0
  hits_best_known: 31

Best run:
  Best length = 39.0
  Best tour   = [9, 1, 10, 12, 13, 2, 0, 11, 6, 15, 5, 14, 4, 3, 8, 7, 16]

--- Running config_3 ---
Summary:
  n: 31
  mean: 39.0
  median: 39.0
  std: 0.0
  min: 39.0
  max: 39.0
  hits_best_known: 31

Best run:
  Best length = 39.0
  Best tour   = [6, 14, 5, 15, 10, 9, 1, 12, 2, 13, 0, 11, 16, 7, 8, 4, 3]


,instance,config,n_cities,mean,median,std,min,max,hits_best_known,best_length,best_tour
0,br17,config_1,17,39.0,39.0,0.0,39.0,39.0,31,39.0,"[5, 15, 14, 6, 3, 4, 8, 16, 7, 9, 10, 1, 12, 2..."
1,br17,config_2,17,39.0,39.0,0.0,39.0,39.0,31,39.0,"[9, 1, 10, 12, 13, 2, 0, 11, 6, 15, 5, 14, 4, ..."
2,br17,config_3,17,39.0,39.0,0.0,39.0,39.0,31,39.0,"[6, 14, 5, 15, 10, 9, 1, 12, 2, 13, 0, 11, 16,..."


In [11]:
# === RUN FT53 ===

inst_ft53 = next(inst for inst in instances_list if inst["name"].lower() == "ft53")

df_ft53 = run_aco_for_instance(
    instance=inst_ft53,
    configs=configs,
    best_known_dict=BEST_KNOWN,
    n_runs=N_RUNS,
    base_seed=42
)

display(df_ft53)


               RUNNING INSTANCE: FT53

--- Running config_1 ---
Summary:
  n: 31
  mean: 7901.451612903225
  median: 7903.0
  std: 107.06503902135917
  min: 7671.0
  max: 8095.0
  hits_best_known: 0

Best run:
  Best length = 7671.0
  Best tour   = [7, 5, 51, 48, 49, 52, 50, 33, 31, 30, 0, 3, 2, 17, 16, 15, 37, 35, 40, 36, 10, 12, 14, 13, 11, 1, 41, 43, 42, 46, 45, 44, 34, 32, 26, 25, 27, 29, 28, 24, 22, 47, 23, 20, 21, 39, 38, 4, 19, 18, 6, 9, 8]

--- Running config_2 ---
Summary:
  n: 31
  mean: 7691.129032258064
  median: 7645.0
  std: 224.3859386467111
  min: 7294.0
  max: 8096.0
  hits_best_known: 0

Best run:
  Best length = 7294.0
  Best tour   = [7, 5, 51, 48, 49, 52, 50, 29, 28, 25, 27, 26, 11, 10, 12, 14, 13, 34, 32, 31, 33, 30, 0, 3, 2, 17, 16, 15, 37, 35, 40, 38, 36, 39, 4, 1, 41, 47, 42, 46, 43, 45, 44, 23, 20, 21, 24, 22, 19, 18, 8, 6, 9]

--- Running config_3 ---
Summary:
  n: 31
  mean: 7755.064516129032
  median: 7703.0
  std: 238.697149191728
  min: 7327.0
  max: 823

,instance,config,n_cities,mean,median,std,min,max,hits_best_known,best_length,best_tour
0,ft53,config_1,53,7901.451613,7903.0,107.065039,7671.0,8095.0,0,7671.0,"[7, 5, 51, 48, 49, 52, 50, 33, 31, 30, 0, 3, 2..."
1,ft53,config_2,53,7691.129032,7645.0,224.385939,7294.0,8096.0,0,7294.0,"[7, 5, 51, 48, 49, 52, 50, 29, 28, 25, 27, 26,..."
2,ft53,config_3,53,7755.064516,7703.0,238.697149,7327.0,8232.0,0,7327.0,"[51, 48, 49, 52, 50, 29, 28, 25, 27, 26, 4, 1,..."


In [22]:
# === RUN KRO124 ===

inst_kro124 = next(inst for inst in instances_list if inst["name"].lower() == "kro124p")

df_kro124 = run_aco_for_instance(
    instance=inst_kro124,
    configs=configs,
    best_known_dict=BEST_KNOWN,
    n_runs=N_RUNS,
    base_seed=42
)

display(df_kro124)



               RUNNING INSTANCE: KRO124P

--- Running config_1 ---


KeyboardInterrupt: 

In [15]:
def run_wilcoxon_tests(df_results, configs, verbose=False):
    from itertools import combinations

    wilcoxon_rows = []
    configs_list = list(configs.keys())
    instances = df_results["instance"].unique()

    for inst in instances:

        if verbose:
            print("\n===============================================")
            print(f"        WILCOXON TESTS FOR INSTANCE: {inst.upper()}")
            print("===============================================\n")

        df_inst = df_results[df_results["instance"] == inst]

        for cfg_a, cfg_b in combinations(configs_list, 2):
            sample_a = df_inst[df_inst["config"] == cfg_a]["best_length"].values
            sample_b = df_inst[df_inst["config"] == cfg_b]["best_length"].values

            if len(sample_a) == 0 or len(sample_b) == 0:
                continue

            test = wilcoxon_rank_sum(sample_a, sample_b)
            stat = test["statistic"]
            p_val = test["p_value"]

            interpretation = (
                "SIGNIFICANT (p < 0.05)"
                if p_val < 0.05 else
                "NOT SIGNIFICANT"
            )

            wilcoxon_rows.append({
                "instance": inst,
                "comparison": f"{cfg_a} vs {cfg_b}",
                "statistic": stat,
                "p_value": p_val,
                "interpretation": interpretation,
            })

    df_wilcoxon = pd.DataFrame(wilcoxon_rows)
    df_wilcoxon = df_wilcoxon.sort_values(by=["instance", "comparison"])

    if verbose:
        display(df_wilcoxon)

    return df_wilcoxon



In [18]:
df_wilcoxon_br17 = run_wilcoxon_tests(df_br17, configs, verbose = True)


        WILCOXON TESTS FOR INSTANCE: BR17



,instance,comparison,statistic,p_value,interpretation
0,br17,config_1 vs config_2,0.0,1.0,NOT SIGNIFICANT
1,br17,config_1 vs config_3,0.0,1.0,NOT SIGNIFICANT
2,br17,config_2 vs config_3,0.0,1.0,NOT SIGNIFICANT


In [21]:
df_wilcoxon_ft53 = run_wilcoxon_tests(df_ft53, configs, verbose = True)


        WILCOXON TESTS FOR INSTANCE: FT53



,instance,comparison,statistic,p_value,interpretation
0,ft53,config_1 vs config_2,1.0,0.317311,NOT SIGNIFICANT
1,ft53,config_1 vs config_3,1.0,0.317311,NOT SIGNIFICANT
2,ft53,config_2 vs config_3,-1.0,0.317311,NOT SIGNIFICANT


In [ ]:
df_wilcoxon_kro124 = run_wilcoxon_tests(df_kro124, configs, verbose = True)